In [1]:
def compute_with_mask():
    try:
        # Prompt user for input
        num = int(input("Enter a number (decimal or hex, e.g., 32 or 0x20): "), 0)  # base 0 allows decimal or hex input
        result = num & 0xf
        print(f"{hex(num)} & 0xf = {hex(result)} ({result} in decimal)")
    except ValueError:
        print("Invalid input. Please enter a valid number (e.g., 42 or 0x2A).")

# Run the function
#compute_with_mask()


Read pcap files

In [2]:
'''
from scapy.all import *

packets = rdpcap('/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp-0-0.pcap')

for pkt in packets:
    if IP in pkt:
        tos = pkt[IP].tos
        print(f"Time: {pkt.time}")
        print(f"Src: {pkt[IP].src} -> Dst: {pkt[IP].dst}")
        print(f"ToS: {hex(tos)} (DSCP: {tos >> 2})")
        print('---')
'''

'\nfrom scapy.all import *\n\npackets = rdpcap(\'/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp-0-0.pcap\')\n\nfor pkt in packets:\n    if IP in pkt:\n        tos = pkt[IP].tos\n        print(f"Time: {pkt.time}")\n        print(f"Src: {pkt[IP].src} -> Dst: {pkt[IP].dst}")\n        print(f"ToS: {hex(tos)} (DSCP: {tos >> 2})")\n        print(\'---\')\n'

## Generate a dataframe for packets using tshark

In [3]:
import subprocess
import pandas as pd

cmd = [
    "tshark",
    "-r", "/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp-0-0.pcap", #/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp-1-0.pcap
    "-T", "fields",
    "-e", "frame.number",              # Packet number
    "-e", "frame.time_relative",       # Relative time
    "-e", "ip.src",                    # Source IP
    "-e", "ip.dst",                    # Destination IP
    "-e", "ip.len",                    # Total IP packet length
    "-e", "ip.dsfield.dscp",           # DSCP

    "-e", "ip.dsfield.ecn",            # ECN (optional)
    "-e", "tcp.seq",                   # TCP sequence number
    "-e", "frame.time_delta",          # Inter-frame delay (jitter)
    "-e", "tcp.srcport",       # Source port (if TCP)
    "-e", "tcp.dstport" ,      # Destination port (if TCP)
    "-e", "udp.srcport",       # Source port (if UDP)
    "-e", "udp.dstport",       # Destination port (if UDP)
    "-e", "ip.proto"          # IP protocol (6 for TCP, 17 for UDP, etc.)    
]

result = subprocess.run(cmd, capture_output=True, text=True)
#print(result.stdout)

lines = result.stdout.strip().split('\n')
rows = [line.split('\t') for line in lines]
#Explicit Congestion Notification (ECN)
df_packets = pd.DataFrame(rows, columns=["Packet number", "Relative_time", "Source_IP", "Destination_IP", "Total_IP_packet_length", "DSCP","ECN","TCP_sequence_number","Inter_frame_delay","tcp_src_port","tcp_dst_port","udp_src_port","udp_dst_port","protocol"])
df_packets['Inter_frame_delay'] = df_packets['Inter_frame_delay'].astype(float)

df_packets['DSCP']=df_packets['DSCP'].astype('int')
df_packets['ECN']=df_packets['ECN'].astype('int')
df_packets['TOS'] = df_packets.apply(lambda row: (int(row['DSCP']) << 2) | int(row['ECN']), axis=1)
df_packets['TOS_hex'] = df_packets['TOS'].apply(lambda x: f"0x{x:02x}")




df_packets['flow_id'] = (df_packets['Source_IP'].astype(str) + '-' +
                         df_packets['Destination_IP'].astype(str) + '-' +
                         df_packets['tcp_src_port'].astype(str) + '-' +
                         df_packets['tcp_dst_port'].astype(str) + '-' +
                         df_packets['protocol'].astype(str))

df_packets.head()

,Packet number,Relative_time,Source_IP,Destination_IP,Total_IP_packet_length,DSCP,ECN,TCP_sequence_number,Inter_frame_delay,tcp_src_port,tcp_dst_port,udp_src_port,udp_dst_port,protocol,TOS,TOS_hex,flow_id
0,1,0.000000000,10.1.1.2,10.1.1.1,56,2,0,0,0.000000,49153,8000,,,6,8,0x08,10.1.1.2-10.1.1.1-49153-8000-6
1,2,0.000000000,10.1.1.1,10.1.1.2,56,2,0,0,0.000000,8000,49153,,,6,8,0x08,10.1.1.1-10.1.1.2-8000-49153-6
2,3,0.000046000,10.1.1.2,10.1.1.1,56,2,0,0,0.000046,49154,8001,,,6,8,0x08,10.1.1.2-10.1.1.1-49154-8001-6
3,4,0.000046000,10.1.1.1,10.1.1.2,56,2,0,0,0.000000,8001,49154,,,6,8,0x08,10.1.1.1-10.1.1.2-8001-49154-6
4,5,0.000093000,10.1.1.2,10.1.1.1,56,3,0,0,0.000047,49155,8002,,,6,12,0x0c,10.1.1.2-10.1.1.1-49155-8002-6


In [4]:
df_packets.groupby('TOS_hex')['Inter_frame_delay'].describe().reset_index()

,TOS_hex,count,mean,std,min,25%,50%,75%,max
0,0x00,653.0,9.249617e-07,0.000005,0.0,0.0,0.000000,0.000000,0.000034
1,0x08,8536.0,3.204876e-04,0.000224,0.0,0.0,0.000472,0.000472,0.003951
2,0x0c,8494.0,3.199341e-04,0.000220,0.0,0.0,0.000472,0.000472,0.000472


There was no congestion (ECN=0), there are no 0x0e packets sent (check simulation and the implementation of the socket) (the tos values in the simulation are those of band2= {0x08,0x0a,0x0c,0x0e})

This is just a stat table to evaluate delay per DSCP type of packet

In [5]:
df_packets_DSCP = (
    df_packets.groupby(['DSCP', 'ECN', 'TOS_hex'])
    .size()
    .reset_index(name='count')
)
# Group by both DSCP and ECN
delay_stats = df_packets.groupby(['DSCP', 'ECN'])['Inter_frame_delay'].describe().reset_index()

df_packets_heading = df_packets_DSCP.merge(delay_stats, on=['DSCP', 'ECN'], how='left')


df_packets_heading


,DSCP,ECN,TOS_hex,count_x,count_y,mean,std,min,25%,50%,75%,max
0,0,0,0x00,653,653.0,9.249617e-07,0.000005,0.0,0.0,0.000000,0.000000,0.000034
1,2,0,0x08,8536,8536.0,3.204876e-04,0.000224,0.0,0.0,0.000472,0.000472,0.003951
2,3,0,0x0c,8494,8494.0,3.199341e-04,0.000220,0.0,0.0,0.000472,0.000472,0.000472


In [6]:
df_packets_heading['TOS_hex'].unique() # should output this {0x08,0x0a,0x0c,0x0e}

array(['0x00', '0x08', '0x0c'], dtype=object)

## Flow stats with flow monitor

In [7]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
flowmon_path="/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp_2_nodes.xml"
tree = ET.parse(flowmon_path)  # Replace with your file path
root = tree.getroot()

# Prepare a list to collect parsed flow data
flow_data = []

# Iterate over all <Flow> elements inside <FlowStats>
#---------------nano seconds to second & bytes to Megabytes
for flow in root.find('FlowStats').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    time_first_tx = float(flow.attrib['timeFirstTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_first_rx = float(flow.attrib['timeFirstRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_tx = float(flow.attrib['timeLastTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_rx = float(flow.attrib['timeLastRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    delay_sum = float(flow.attrib['delaySum'].replace('ns', '').replace('+', ''))*1E-9
    jitter_sum = float(flow.attrib['jitterSum'].replace('ns', '').replace('+', ''))*1E-9
    last_delay = float(flow.attrib['lastDelay'].replace('ns', '').replace('+', ''))*1E-9
    tx_bytes = int(flow.attrib['txBytes'])
    rx_bytes = int(flow.attrib['rxBytes'])
    tx_packets = int(flow.attrib['txPackets'])
    rx_packets = int(flow.attrib['rxPackets'])
    lost_packets = int(flow.attrib['lostPackets'])
    times_forwarded = int(flow.attrib['timesForwarded'])
    duration = time_last_rx - time_first_tx
    throughput_Mbps = (int(flow.attrib['rxBytes']) * 8) / (duration * 1e6)  # bits/sec → Mbps


    flow_data.append({
        'flow_id': flow_id,
        'time_first_tx_s': time_first_tx,
        'time_first_rx_s': time_first_rx,
        'time_last_tx_s': time_last_tx,
        'time_last_rx_s': time_last_rx,
        'delay_sum_s': delay_sum,
        'jitter_sum_s': jitter_sum,
        'last_delay_s': last_delay,
        'tx_bytes': tx_bytes,
        'rx_bytes': rx_bytes,
        'tx_packets': tx_packets,
        'rx_packets': rx_packets,
        'lost_packets': lost_packets,
        'times_forwarded': times_forwarded,
        'duration':duration,
        'throughput_Mbps':throughput_Mbps
    })

classifier_info = {}
for flow in root.find('Ipv4FlowClassifier').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    classifier_info[flow_id] = {
        'src': flow.attrib['sourceAddress'],
        'dst': flow.attrib['destinationAddress'],
        'src_port': flow.attrib['sourcePort'],
        'dst_port': flow.attrib['destinationPort'],
        'protocol': flow.attrib['protocol']
    }

# Merge with previous stats if needed
for row in flow_data:
    fid = row['flow_id']
    if fid in classifier_info:
        row.update(classifier_info[fid])
df_flowmon = pd.DataFrame(flow_data)
df_flowmon['mean_delay']=  df_flowmon['delay_sum_s']/df_flowmon['rx_packets']
df_flowmon['mean-jitter']=df_flowmon['jitter_sum_s']/df_flowmon['rx_packets']

df_flowmon['flow_5tuple'] = (df_flowmon['src'].astype(str) + '-' +
                         df_flowmon['dst'].astype(str) + '-' +
                         df_flowmon['src_port'].astype(str) + '-' + 
                         df_flowmon['dst_port'].astype(str) + '-' +
                         df_flowmon['protocol'].astype(str))
df_flowmon.head()

,flow_id,time_first_tx_s,time_first_rx_s,time_last_tx_s,time_last_rx_s,delay_sum_s,jitter_sum_s,last_delay_s,tx_bytes,rx_bytes,...,duration,throughput_Mbps,src,dst,src_port,dst_port,protocol,mean_delay,mean-jitter,flow_5tuple
0,1,1.00000,1.00205,6.05340,6.36190,539.54100,2.108090,0.313213,1701668,1701668,...,5.36190,2.538903,10.1.1.2,10.1.1.1,49153,8000,6,0.185920,7.264266e-04,10.1.1.2-10.1.1.1-49153-8000-6
1,2,1.00000,1.00209,6.09319,6.39789,542.84000,2.113560,0.309429,1701668,1701668,...,5.39789,2.521975,10.1.1.2,10.1.1.1,49154,8001,6,0.187057,7.283115e-04,10.1.1.2-10.1.1.1-49154-8001-6
2,3,1.00000,1.00214,6.13297,6.43389,545.33200,2.120140,0.305645,1701668,1701668,...,5.43389,2.505267,10.1.1.2,10.1.1.1,49155,8002,6,0.187916,7.305789e-04,10.1.1.2-10.1.1.1-49155-8002-6
3,4,1.00000,1.00219,6.17086,6.45585,539.00200,5.150370,0.291598,1684260,1684260,...,5.45585,2.469657,10.1.1.2,10.1.1.1,49156,8003,6,0.187544,1.792056e-03,10.1.1.2-10.1.1.1-49156-8003-6
4,5,1.00205,1.00409,6.36190,6.36393,3.12456,0.000013,0.002034,77644,77644,...,5.36188,0.115846,10.1.1.1,10.1.1.2,8000,49153,6,0.002042,8.366013e-09,10.1.1.1-10.1.1.2-8000-49153-6


In [8]:
df_flowmon.columns

Index(['flow_id', 'time_first_tx_s', 'time_first_rx_s', 'time_last_tx_s',
       'time_last_rx_s', 'delay_sum_s', 'jitter_sum_s', 'last_delay_s',
       'tx_bytes', 'rx_bytes', 'tx_packets', 'rx_packets', 'lost_packets',
       'times_forwarded', 'duration', 'throughput_Mbps', 'src', 'dst',
       'src_port', 'dst_port', 'protocol', 'mean_delay', 'mean-jitter',
       'flow_5tuple'],
      dtype='object')

In [9]:
df_flowmon[['tx_bytes', 'rx_bytes', 'tx_packets', 'rx_packets', 'lost_packets',
       'times_forwarded', 'duration', 'throughput_Mbps','mean_delay', 'mean-jitter', 'src', 'dst',
       'src_port', 'dst_port']].head()

,tx_bytes,rx_bytes,tx_packets,rx_packets,lost_packets,times_forwarded,duration,throughput_Mbps,mean_delay,mean-jitter,src,dst,src_port,dst_port
0,1701668,1701668,2902,2902,0,0,5.36190,2.538903,0.185920,7.264266e-04,10.1.1.2,10.1.1.1,49153,8000
1,1701668,1701668,2902,2902,0,0,5.39789,2.521975,0.187057,7.283115e-04,10.1.1.2,10.1.1.1,49154,8001
2,1701668,1701668,2902,2902,0,0,5.43389,2.505267,0.187916,7.305789e-04,10.1.1.2,10.1.1.1,49155,8002
3,1684260,1684260,2874,2874,0,0,5.45585,2.469657,0.187544,1.792056e-03,10.1.1.2,10.1.1.1,49156,8003
4,77644,77644,1530,1530,0,0,5.36188,0.115846,0.002042,8.366013e-09,10.1.1.1,10.1.1.2,8000,49153


In [10]:
#df_flowmon[['rx_packets']].sum()

In [11]:
df_flowmon[['tx_packets','rx_packets','lost_packets',\
            'times_forwarded','duration','throughput_Mbps',	'mean_delay','mean-jitter']].describe()

,tx_packets,rx_packets,lost_packets,times_forwarded,duration,throughput_Mbps,mean_delay,mean-jitter
count,8.000000,8.000000,8.0,8.0,8.000000,8.000000,8.000000,8.000000e+00
mean,2210.375000,2210.375000,0.0,0.0,5.412343,1.311710,0.094576,4.972185e-04
std,731.973446,731.973446,0.0,0.0,0.038186,1.280051,0.098924,6.353329e-04
min,1513.000000,1513.000000,0.0,0.0,5.361880,0.112645,0.002042,8.366013e-09
25%,1530.000000,1530.000000,0.0,0.0,5.388855,0.114884,0.002042,1.071895e-07
50%,2202.000000,2202.000000,0.0,0.0,5.415840,1.292752,0.093981,3.633032e-04
75%,2902.000000,2902.000000,0.0,0.0,5.439343,2.509444,0.187179,7.288784e-04
max,2902.000000,2902.000000,0.0,0.0,5.455850,2.538903,0.187916,1.792056e-03


In [12]:
'''
import xml.etree.ElementTree as ET

# Load the XML file

# Navigate to the Ipv4FlowClassifier
classifier = root.find("Ipv4FlowClassifier")

# Extract all Flow entries
for flow in classifier.findall("Flow"):
    flow_id = flow.get("flowId")
    src_addr = flow.get("sourceAddress")
    dst_addr = flow.get("destinationAddress")
    src_port = flow.get("sourcePort")
    dst_port = flow.get("destinationPort")

    print(f"\nFlow ID {flow_id} | {src_addr}:{src_port} → {dst_addr}:{dst_port}")

    # For each DSCP under this Flow
    for dscp in flow.findall("Dscp"):
        dscp_value = dscp.get("value")
        packets = dscp.get("packets")
        print(f"  DSCP {dscp_value} → {packets} packets")
'''


'\nimport xml.etree.ElementTree as ET\n\n# Load the XML file\n\n# Navigate to the Ipv4FlowClassifier\nclassifier = root.find("Ipv4FlowClassifier")\n\n# Extract all Flow entries\nfor flow in classifier.findall("Flow"):\n    flow_id = flow.get("flowId")\n    src_addr = flow.get("sourceAddress")\n    dst_addr = flow.get("destinationAddress")\n    src_port = flow.get("sourcePort")\n    dst_port = flow.get("destinationPort")\n\n    print(f"\nFlow ID {flow_id} | {src_addr}:{src_port} → {dst_addr}:{dst_port}")\n\n    # For each DSCP under this Flow\n    for dscp in flow.findall("Dscp"):\n        dscp_value = dscp.get("value")\n        packets = dscp.get("packets")\n        print(f"  DSCP {dscp_value} → {packets} packets")\n'

Note: the file qdisc_dscp_counts.txt might be reducdant as the same information can be gathered from the flowmonitor .xml file

In [13]:
import pandas as pd

data = []
with open("/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp_counts.txt", "r") as f:
    for line in f:
        parts = line.strip().split()
        FLowID = int(parts[0].split('=')[1])
        dscp = int(parts[1].split('=')[1], 16) #read in hexadecimal format
        count = int(parts[2].split('=')[1])
     
        data.append({"FlowID":FLowID,"dscp": hex(dscp), "count": count})

df_dscp = pd.DataFrame(data)
df_dscp


,FlowID,dscp,count
0,1,0x2,968
1,1,0x0,1
2,2,0x2,960
3,2,0x0,2
4,3,0x3,934
...,...,...,...
331,6,0x0,160
332,7,0x3,1370
333,7,0x0,160
334,8,0x3,1358


In [14]:
df_dscp.groupby('dscp')[['count']].sum()

,count
dscp,
0x0,8154
0x2,110694
0x3,109779


## Identify packets to their corresponding flow (works only for one protocol)

In [15]:
set(df_packets['flow_id'].unique())==set(df_flowmon['flow_5tuple'].unique())

True

Verify that the number of packets per flow is correct

In [16]:
df_flowmon[['flow_5tuple','rx_packets']]

,flow_5tuple,rx_packets
0,10.1.1.2-10.1.1.1-49153-8000-6,2902
1,10.1.1.2-10.1.1.1-49154-8001-6,2902
2,10.1.1.2-10.1.1.1-49155-8002-6,2902
3,10.1.1.2-10.1.1.1-49156-8003-6,2874
4,10.1.1.1-10.1.1.2-8000-49153-6,1530
5,10.1.1.1-10.1.1.2-8001-49154-6,1530
6,10.1.1.1-10.1.1.2-8002-49155-6,1530
7,10.1.1.1-10.1.1.2-8003-49156-6,1513


In [17]:
df_packets['flow_id'].value_counts().head()

flow_id
10.1.1.2-10.1.1.1-49153-8000-6    2902
10.1.1.2-10.1.1.1-49154-8001-6    2902
10.1.1.2-10.1.1.1-49155-8002-6    2902
10.1.1.2-10.1.1.1-49156-8003-6    2874
10.1.1.1-10.1.1.2-8000-49153-6    1530
Name: count, dtype: int64

## Queues events

In [18]:
import re
import pandas as pd

# Read lines from log file
with open("/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp_trace.log", "r") as f:
    lines = f.readlines()

# Initialize empty list for parsed entries
data = []

# Regex pattern to capture all line types
pattern = re.compile(r'(?P<time>[\d.]+)s: (?P<type>\w+) (?P<from>[\d.]+) to (?P<to>[\d.]+)|(?P<time2>[\d.]+)s: SojournTime (?P<sojourn>[\d.]+) ms')

# Parse lines
for line in lines:
    match = pattern.match(line.strip())
    if match:
        if match.group('type') == 'DevicePacketsInQueue' or match.group('type') == 'TcPacketsInQueue':
            data.append({
                'time': float(match.group('time')),
                'event': match.group('type'),
                'from': int(match.group('from')),
                'to': int(match.group('to'))
            })
        elif match.group('sojourn'):
            data.append({
                'time': float(match.group('time2')),
                'event': 'SojournTime',
                'value_ms': float(match.group('sojourn'))
            })

# Convert to pandas DataFrame
df_events = pd.DataFrame(data)

#df.iloc[20:30]


In [19]:
df_events.head()

,time,event,from,to,value_ms
0,1.0,TcPacketsInQueue,0.0,1.0,NaN
1,1.0,TcPacketsInQueue,1.0,0.0,NaN
2,1.0,SojournTime,NaN,NaN,0.0
3,1.0,DevicePacketsInQueue,0.0,1.0,NaN
4,1.0,DevicePacketsInQueue,1.0,0.0,NaN


## Application-level logs

In [20]:
import pandas as pd
import re

rows = []
with open("/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/qdisc_dscp_sink_log.txt") as f:
    for line in f:
        match = re.search(r'\+([\d.]+)s sink received (\d+) bytes from ([\d.]+) port (\d+)', line)
        if match:
            time, size, ip, port = match.groups()
            rows.append((float(time), int(size), ip, int(port)))

df_sink = pd.DataFrame(rows, columns=['Time', 'Size', 'Source_IP', 'Port'])
df_sink.head()


,Time,Size,Source_IP,Port
0,1.00670,536,10.1.1.2,49153
1,1.00717,536,10.1.1.2,49153
2,1.00752,376,10.1.1.2,49153
3,1.00803,536,10.1.1.2,49154
4,1.00850,536,10.1.1.2,49154
